# Reddit Data Scraper

### Import Libraries

In [2]:
import os
import time
import datetime
import pandas as pd
import praw
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
from io import StringIO
from datetime import datetime
from typing import List, Dict, Any, Optional, Tuple
import numpy as np
from tqdm import tqdm
import json

## First let's get S&P 500 Tickers from Wikipedia. This way we have a scope for our analysis

In [10]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
response = requests.get(url)
soup = BeautifulSoup(response.text)
table = soup.find("table", {"class": "wikitable"})
sp500 = pd.read_html(StringIO(str(table)))[0]
sp500.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


# Reddit API Client Initialization
First let's load the api environment to collect all the necessary data. 
The idea is to download posts and comments from different subreddits. 
- Subreddits: 'wallstreetbets', 'stocks', 'investing', 'StockMarket'
- General Search & Stock Specific search

In [3]:
# First we will load the environment
load_dotenv('api.env')
# Import the ids and secret keys from our environment
id = os.getenv('REDDIT_CLIENT_ID')
secret = os.getenv('REDDIT_CLIENT_SECRET')
agent = os.getenv('REDDIT_USER_AGENT', 'StockDataScraper v1.0')
reddit = praw.Reddit(client_id = id,
                     client_secret = secret,
                     user_agent = agent)

## Let's now collect all of the data
#### Data Collection Function

In [ ]:
def get_reddit_data(reddit, subreddit_name, data_type='posts', search_term=None, 
                   time_filter='year', limit=200, comment_limit=30):
    """This is the general function that I will loop through in order to download
      all the Reddit Data"""
    
    subreddit = reddit.subreddit(subreddit_name)
    posts_list = []
    comments_list = []
    
    try:
        # Determine which data to fetch
        if data_type == 'search' and search_term:
            print(f"Searching for '{search_term}' in r/{subreddit_name}...")
            posts = subreddit.search(search_term, limit=limit)
            search_keywords = [search_term.lower(), f"${search_term.lower()}"]
        else:
            print(f"Getting top posts from r/{subreddit_name} for {time_filter}...")
            posts = subreddit.top(time_filter=time_filter, limit=limit)
            search_keywords = None
            
        # Process posts
        for i, post in enumerate(posts):
            # Filter search results if needed
            if search_keywords and not any(kw in (post.title + " " + post.selftext).lower() for kw in search_keywords):
                continue
                
            # Extract post data
            post_data = {
                'post_id': post.id,
                'title': post.title,
                'selftext': post.selftext,
                'score': post.score,
                'upvote_ratio': post.upvote_ratio,
                'created_utc': datetime.datetime.fromtimestamp(post.created_utc),
                'num_comments': post.num_comments,
                'author': str(post.author),
                'permalink': post.permalink,
                'url': post.url,
                'is_self': post.is_self,
                'flair': post.link_flair_text,
                'subreddit': subreddit_name,
                'category': 'stock_specific' if data_type == 'search' else 'general'
            }
            
            # Add search term if applicable
            if search_term:
                post_data['search_term'] = search_term
                
            posts_list.append(post_data)
            
            # Get comments
            try:
                post.comments.replace_more(limit=0)
                for comment in post.comments.list()[:comment_limit]:
                    # Filter comments for search terms if needed
                    if search_keywords and not any(kw in comment.body.lower() for kw in search_keywords):
                        continue
                        
                    comment_data = {
                        'comment_id': comment.id,
                        'post_id': post.id,
                        'parent_id': comment.parent_id,
                        'body': comment.body,
                        'score': comment.score,
                        'created_utc': datetime.datetime.fromtimestamp(comment.created_utc),
                        'author': str(comment.author),
                        'subreddit': subreddit_name,
                        'category': 'stock_specific' if data_type == 'search' else 'general'
                    }
                    
                    # Add search
                    if search_term:
                        comment_data['search_term'] = search_term
                        
                    comments_list.append(comment_data)
            except Exception as e:
                print(f"Error processing comments for post {post.id}: {e}")
                
            # Be nice to Reddit's servers
                
        print(f"Found {len(posts_list)} posts and {len(comments_list)} comments")
        return pd.DataFrame(posts_list) if posts_list else pd.DataFrame(), \
               pd.DataFrame(comments_list) if comments_list else pd.DataFrame()
               
    except Exception as e:
        print(f"Error fetching data from r/{subreddit_name}: {e}")
        return pd.DataFrame(), pd.DataFrame()

#### Here we are downloading General top Posts from the following subreddits. These posts are not stock specific rather, they are the top posts a user might see when looking at the following subreddits for the past year

In [29]:
# Get S&P 500 tickers
sp500_tickers = sp500['Symbol'].tolist()
# List of finance subreddits
subreddits = ['Economics','finance','wallstreetbets', 'stocks', 'investing']
# Initialize aggregated DataFrames

all_posts = pd.DataFrame()
all_comments = pd.DataFrame()

# Loop through all the subreddits defined above and collect the posts and comments data
for subreddit in subreddits:
    posts_df, comments_df = get_reddit_data(reddit, subreddit,data_type='posts',
                                            limit=200,comment_limit=30)
    all_posts = pd.concat([all_posts, posts_df],ignore_index=True)
    all_comments = pd.concat([all_comments, comments_df],ignore_index=True)
    time.sleep(1)

Getting top posts from r/Economics for year...
Found 200 posts and 5935 comments
Getting top posts from r/finance for year...
Found 100 posts and 1870 comments
Getting top posts from r/wallstreetbets for year...
Found 200 posts and 6000 comments
Getting top posts from r/stocks for year...
Found 200 posts and 5985 comments
Getting top posts from r/investing for year...
Found 200 posts and 5876 comments


In [31]:
# Safe all posts and comments into our file
all_posts.to_csv('./data/reddit/general_posts.csv',index = False)
all_comments.to_csv('./data/reddit/general_comments.csv',index = False)

#### Let's now use the same function to collect stock specific reddit data

In here we don't want to scrape data for all of the subreddits but only keeping the few popular ones. However, we will download for all the stocks in the S&P500
I will chose r/stocks and r/wallstreetbets due to it's recent popularity

In [32]:
subreddits = ['stocks','wallstreetbets']
all_posts = pd.DataFrame()
all_comments = pd.DataFrame()
for subreddit in subreddits:
    for ticker in sp500_tickers:
        posts_df , comments_df = get_reddit_data(
            reddit, subreddit, data_type='search',search_term=ticker,
            limit=50, comment_limit=20
        )
        all_posts= pd.concat([all_posts, posts_df],ignore_index = True)
        all_comments = pd.concat([all_comments,comments_df],ignore_index=True)
        time.sleep(1)    

Searching for 'MMM' in r/stocks...
Found 37 posts and 48 comments
Searching for 'AOS' in r/stocks...
Found 14 posts and 1 comments
Searching for 'ABT' in r/stocks...
Found 39 posts and 17 comments
Searching for 'ABBV' in r/stocks...
Found 43 posts and 91 comments
Searching for 'ACN' in r/stocks...
Found 46 posts and 3 comments
Searching for 'ADBE' in r/stocks...
Found 46 posts and 56 comments
Searching for 'AMD' in r/stocks...
Found 50 posts and 400 comments
Searching for 'AES' in r/stocks...
Found 23 posts and 6 comments
Searching for 'AFL' in r/stocks...
Found 18 posts and 5 comments
Searching for 'A' in r/stocks...
Found 50 posts and 919 comments
Searching for 'APD' in r/stocks...
Found 21 posts and 16 comments
Searching for 'ABNB' in r/stocks...
Found 40 posts and 74 comments
Searching for 'AKAM' in r/stocks...
Found 39 posts and 15 comments
Searching for 'ALB' in r/stocks...
Found 32 posts and 43 comments
Searching for 'ARE' in r/stocks...
Found 50 posts and 278 comments
Searching

In [33]:
all_posts.to_csv('./data/reddit/posts_stock_specific.csv')
all_comments.to_csv('./data/reddit/comments_stock_specific.csv')

# Financial Data Collector

In [25]:
# Create data directory
data_dir = "./data/financial_data"
os.makedirs(data_dir, exist_ok=True)

print("Setup complete!")

Setup complete!


### API configuration & Setting up ticker to cik json file to download from edgar based on cik value with leading zeros

In [26]:
# IMPORTANT: Replace with your actual information as required by SEC


# Set my identity for the SEC (First name last name and email)
name = "Maseeh"
surname = "Faizan"
email = "maseehfaizan@gmail.com"
headers = {'User-Agent': f'{name} {surname} {email}'}

# Get ticker data
ticker = requests.get('https://www.sec.gov/files/company_tickers.json', headers=headers).json()
ticker_df = pd.DataFrame.from_dict(ticker, orient='index')
ticker_df.rename(columns={'cik_str':'cik','title':'name'}, inplace=True)

# Fill in the cik code and add the leading zeros
ticker_df['cik'] = ticker_df['cik'].astype(str).str.zfill(10)

print(f"Loaded ticker data for {len(ticker_df)} companies")
print("Sample ticker data:")


# API endpoints
COMPANY_FACTS_URL = "https://data.sec.gov/api/xbrl/companyfacts/CIK{}.json"

print("API configuration set!")

Loaded ticker data for 10046 companies
Sample ticker data:
API configuration set!


In [27]:
# Variables to track progress
processed_companies = []
failed_companies = []
all_raw_data = []

# Processing parameters
delay_between_requests = 0.5  # seconds
max_retries = 3

print("Ready to start downloading!")

Ready to start downloading!


In [28]:
# Create a helper function to get CIK from ticker using your ticker_df
def get_cik_from_ticker(symbol):
    """Get CIK for a given ticker symbol from ticker_df"""
    result = ticker_df[ticker_df['ticker'] == symbol.upper()]
    if not result.empty:
        return result.iloc[0]['cik']
    else:
        return None

### Main Downloading function

In [ ]:
company_symbols = sp500['Symbol'].tolist()
for i, symbol in enumerate(tqdm(company_symbols, desc="Downloading financial data")):
    print(f"\nProcessing {symbol} ({i+1}/{len(company_symbols)})")
    
    # Get CIK for the symbol using your ticker_df
    cik = get_cik_from_ticker(symbol)
    
    if not cik:
        print(f"Could not find CIK for {symbol}")
        failed_companies.append(symbol)
        continue
    
    print(f"Found CIK: {cik} for {symbol}")
    
    # Download company facts with retries
    facts_data = None
    for attempt in range(max_retries):
        try:
            url = COMPANY_FACTS_URL.format(cik)
            response = requests.get(url, headers=headers)
            
            if response.status_code == 429:
                print("Rate limit hit, waiting 10 seconds...")
                time.sleep(10)
                response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                facts_data = response.json()
                break
            else:
                print(f"Attempt {attempt+1}: Status code {response.status_code}")
                
        except Exception as e:
            print(f"Attempt {attempt+1} failed: {e}")
        
        if attempt < max_retries - 1:
            time.sleep(delay_between_requests * (2 ** attempt))
    
    if not facts_data:
        print(f"Failed to get data for {symbol} after {max_retries} attempts")
        failed_companies.append(symbol)
        continue
    
    # Extract raw financial data
    if 'facts' in facts_data and 'us-gaap' in facts_data['facts']:
        company_name = facts_data.get('entityName', 'Unknown')
        print(f"Processing data for {company_name}")
        
        us_gaap_facts = facts_data['facts']['us-gaap']
        
        company_records = []
        for concept, concept_data in us_gaap_facts.items():
            units = concept_data.get('units', {})
            
            for unit_type, values in units.items():
                for value_data in values:
                    record = {
                        'symbol': symbol,
                        'cik': cik,
                        'company_name': company_name,
                        'concept': concept,
                        'unit': unit_type,
                        'start': value_data.get('start', None),
                        'end': value_data.get('end', None),
                        'filed': value_data.get('filed', None),
                        'form': value_data.get('form', None),
                        'frame': value_data.get('frame', None),
                        'value': value_data.get('val', None)
                    }
                    company_records.append(record)
        
        if company_records:
            # Convert to DataFrame and save individual company file
            company_df = pd.DataFrame(company_records)
            
            # Convert dates
            for date_col in ['start', 'end', 'filed']:
                company_df[date_col] = pd.to_datetime(company_df[date_col], errors='coerce')
            
            # Save individual company raw data
            company_file = os.path.join(data_dir, f"{symbol}_raw_financials.csv")
            company_df.to_csv(company_file, index=False)
            
            # Add to master list
            all_raw_data.extend(company_records)
            processed_companies.append(symbol)
            
            print(f"Saved {len(company_records)} records for {symbol}")
        else:
            print(f"No financial data found for {symbol}")
            failed_companies.append(symbol)
    else:
        print(f"No US-GAAP facts found for {symbol}")
        failed_companies.append(symbol)
    
    # Wait between requests
    time.sleep(delay_between_requests)

print(f"\nDownload complete!")
print(f"Successfully processed: {len(processed_companies)} companies")
print(f"Failed: {len(failed_companies)} companies")


Processing MMM (1/503)
Found CIK: 0000066740 for MMM
Processing data for 3M COMPANY
Saved 33120 records for MMM



Processing AOS (2/503)
Found CIK: 0000091142 for AOS
Processing data for A. O. Smith Corporation
Saved 19078 records for AOS



Processing ABT (3/503)
Found CIK: 0000001800 for ABT
Processing data for ABBOTT LABORATORIES
Saved 21203 records for ABT



Processing ABBV (4/503)
Found CIK: 0001551152 for ABBV
Processing data for AbbVie Inc.
Saved 18538 records for ABBV



Processing ACN (5/503)
Found CIK: 0001467373 for ACN
Processing data for Accenture plc
Saved 27113 records for ACN


...


Processing YUM (500/503)
Found CIK: 0001041061 for YUM
Processing data for YUM! BRANDS, INC.
Saved 21980 records for YUM



Processing ZBRA (501/503)
Found CIK: 0000877212 for ZBRA
Processing data for Zebra Technologies Corporation
Saved 27171 records for ZBRA



Processing ZBH (502/503)
Found CIK: 0001136869 for ZBH
Processing data for ZIMMER BIOMET HOLDINGS, INC.
Saved 25638 records for ZBH



Processing ZTS (503/503)
Found CIK: 0001555280 for ZTS
Processing data for Zoetis Inc.
Saved 21406 records for ZTS



Download complete!
Successfully processed: 501 companies
Failed: 2 companies


# This section is to parse text data from the HTML format and categorize each section


The idea here is First I will find any text that has the starting pattern (i.e Item 1 Business). It also needs to have the ending pattern i.e Item 1A. When that is the case then the text in between is the text that is part of Item 1. This approach works for most of the companies tested.
Same for all the other parts.

I am also making sure the longest text is the main text, (This way it ignores table of contents or other anomalies)

In [ ]:
import os
import requests
import pandas as pd
import time

# Headers for SEC API requests
headers = {
    'User-Agent': 'Maseeh Faizan maseeh.faizan@unil.ch'
}

# Base URL for SEC data
base_url = "https://data.sec.gov"

# Create directory for storing filings
os.makedirs('./data/sec_filings', exist_ok=True)

# Loop through all companies in the dataframe
for _, row in sp500.iterrows():
    company = row['Symbol']
    cik = row['CIK']

    print(f"Processing {company} (CIK: {cik})...")

    try:
        # Get company submissions
        submissions_url = f"{base_url}/submissions/CIK{cik:010d}.json"
        response = requests.get(submissions_url, headers=headers)
        response.raise_for_status()
        submissions = response.json()

        # Convert to DataFrame
        submissions_df = pd.DataFrame(submissions['filings']['recent'])

        # Filter for 10-K filings
        submissions_df = submissions_df[submissions_df['form'] == '10-K']

        # Create document URLs
        submissions_df['doc_url'] = submissions_df.apply(
            lambda x: f"https://www.sec.gov/Archives/edgar/data/{cik}/{x['accessionNumber'].replace('-', '')}/{x['primaryDocument']}",
            axis=1
        )

        # Skip if no 10-K filings found
        if len(submissions_df) == 0:
            print(f"No 10-K filings found for {company}")
            continue

        # Download each filing
        for idx, filing_row in submissions_df.iterrows():
            url = filing_row['doc_url']
            form = filing_row['form']
            date = filing_row['reportDate']

            # Create filename with company symbol
            filename = f"{company}_{form}_{date}.html"
            file_path = os.path.join('sec_filings', filename)

            try:
                response = requests.get(url, headers=headers)
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(response.text)
                print(f"Downloaded: {filename}")
            except Exception as e:
                print(f"Error downloading {filename}: {e}")

        print(f"Completed {company} ({len(submissions_df)} filings)")

        # Add a small delay to be respectful of SEC's servers
        time.sleep(0.1)

    except Exception as e:
        print(f"Error processing {company}: {e}")

print("All companies processed!")